In [11]:
id = "8475883"
jsonJoueur = requests.get("https://statsapi.web.nhl.com/api/v1/people/" + id + "/stats?stats=gameLog&season=20202021").json()
jsonJoueur

{'copyright': 'NHL and the NHL Shield are registered trademarks of the National Hockey League. NHL and NHL team marks are the property of the NHL and its teams. © NHL 2021. All Rights Reserved.',
 'stats': [{'type': {'displayName': 'gameLog', 'gameType': None},
   'splits': [{'season': '20202021',
     'stat': {'timeOnIce': '59:58',
      'ot': 0,
      'shutouts': 0,
      'saves': 27,
      'powerPlaySaves': 5,
      'shortHandedSaves': 1,
      'evenSaves': 21,
      'shortHandedShots': 1,
      'evenShots': 21,
      'powerPlayShots': 6,
      'decision': 'W',
      'savePercentage': 0.964,
      'games': 1,
      'gamesStarted': 1,
      'shotsAgainst': 28,
      'goalsAgainst': 1,
      'powerPlaySavePercentage': 83.33333333333334,
      'shortHandedSavePercentage': 100.0,
      'evenStrengthSavePercentage': 100.0},
     'team': {'id': 10,
      'name': 'Toronto Maple Leafs',
      'link': '/api/v1/teams/10'},
     'opponent': {'id': 23,
      'name': 'Vancouver Canucks',
      '